In [1]:
import pandas as pd
import numpy as np
import spacy
import re

In [2]:
df = pd.read_csv('titles-and-genres.csv',index_col=0)
df.head()

,title,genre,description,review,rating
0,And Then There Were None,Mystery,"First, there were ten—a curious assortment of ...",There is scarcely any comfort to be found in t...,4.0
1,Murder on the Orient Express,Mystery,"Just after midnight, a snowdrift stops the fam...",I’m pretty astounded by how much fun this was....,4.0
2,The Girl with the Dragon Tattoo,Mystery,"Harriet Vanger, a scion of one of Sweden’s wea...",I've heard and read many complaints about Stie...,4.0
3,Gone Girl,Mystery,This is an alternate cover edition for ISBN 97...,3 1/2 stars.This is going to be a hard review ...,3.0
4,The Girl on the Train,Mystery,Rachel catches the same commuter train every m...,I once read a book by a former alcoholic where...,4.0


In [3]:
df.genre.value_counts()

genre
Action             959
Science-fiction    500
Thriller           498
Mystery            497
Fiction            490
Autobiography      490
Romance            479
Fantasy            478
Motivational       470
Name: count, dtype: int64

In [136]:
# df['genre'] = df['genre'].apply(lambda x:'Romance' if x=='M M Romance' else x)
# df['genre'] = df['genre'].apply(lambda x:'Fiction' if x=='Magic' or x=='Christian Fiction' or x=='Historical Fiction' else x)
# df['genre'] = df['genre'].apply(lambda x:'Fantasy' if x=='Urban Fantasy' else x)
# df['genre'] = df['genre'].apply(lambda x:'Horror' if x=='Paranormal' else x)
# df['genre'] = df['genre'].apply(lambda x:'Science Fiction' if x=='Time Travel' else x)

In [137]:
# genres = list(df.genre.value_counts().index[:10])

In [138]:
# df['genre'] = df['genre'].apply(lambda x:x if x in genres else 'Other')

In [139]:
# df = df.query('genre != "Other"')

In [140]:
# df.genre.value_counts()

In [4]:
df['rating'].value_counts()

rating
5.0    1769
4.0    1005
1.0     869
2.0     718
3.0     500
Name: count, dtype: int64

In [5]:
nlp = spacy.load('en_core_web_lg')

In [6]:
df['review'] = df['review'].apply(lambda x:re.sub(r'[^\w\s]','', x))

In [7]:
df['description'] = df['description'].apply(lambda x:re.sub(r'[^\w\s]','', x))

In [12]:
from tqdm import tqdm
tqdm.pandas()
df['embeddings_description'] = df['description'].progress_apply(lambda x: nlp(x).vector)
#df['embeddings_review'] = df['review'].progress_apply(lambda x: nlp(x).vector)

100%|██████████| 4861/4861 [02:12<00:00, 36.81it/s]


In [146]:
df['embeddings_title'] = df['title'].progress_apply(lambda x: nlp(x).vector)

100%|██████████| 4861/4861 [00:18<00:00, 258.36it/s]


In [147]:
df['rating']=df['rating'].astype(int)

In [148]:
df.shape

(4861, 6)

In [3]:
df_emb = pd.read_csv('books-embeddings.csv',index_col=0)
df_emb.head()

,title,genre,description,review,rating,embeddings_description,embeddings_review
0,And Then There Were None,Mystery,First there were tena curious assortment of st...,There is scarcely any comfort to be found in t...,4,[-2.3472178e+00 1.1913711e+00 -1.7946842e+00 ...,[-1.63397443e+00 1.04347742e+00 -2.35051107e+...
1,Murder on the Orient Express,Mystery,Just after midnight a snowdrift stops the famo...,Im pretty astounded by how much fun this was I...,4,[-2.31025434e+00 1.31847060e+00 -1.37480509e+...,[-1.04789340e+00 1.73862159e+00 -2.57041287e+...
2,The Girl with the Dragon Tattoo,Mystery,Harriet Vanger a scion of one of Swedens wealt...,Ive heard and read many complaints about Stieg...,4,[-2.0903664 0.2743508 -1.6205834 -0.405481...,[-1.6553577 1.3631548 -2.0955274 -0.733144...
3,Gone Girl,Mystery,This is an alternate cover edition for ISBN 97...,3 12 starsThis is going to be a hard review to...,3,[-1.4051157 1.447949 -1.8702524 -0.349049...,[-0.98408294 1.775969 -2.7781804 -1.217786...
4,The Girl on the Train,Mystery,Rachel catches the same commuter train every m...,I once read a book by a former alcoholic where...,4,[-1.0186274 2.239291 -2.2797942 -0.611633...,[-1.4940541e+00 1.2996707e+00 -2.5178683e+00 ...


In [8]:
df['sentiment'] = df['rating'].apply(lambda x:1 if x==5 or x==4 else -1 if x==1 or x==2 else 0)

In [9]:
df['sentiment'].value_counts()

sentiment
 1    2774
-1    1587
 0     500
Name: count, dtype: int64

In [10]:
df.dropna(inplace=True)

In [82]:
test = df.iloc[-1000:,:]

In [83]:
test

,title,genre,description,review,rating,embeddings_description,embeddings_review
3954,The Last Olympian,Action,All year the halfbloods have been preparing fo...,Wow that was a great ending to this very fanta...,5,"[-1.9361616, 0.9021592, -1.2837694, 1.1761153,...","[-0.86090416, 1.1178876, -2.989706, -1.4590447..."
3955,The Bourne Identity,Action,Jason Bourne does not know the answer to any o...,First of all let me saythe recent movie with M...,5,"[-1.6277697, 2.9264565, -1.2738069, -0.2580757...","[-1.608945, 1.0053965, -2.124535, -0.9773431, ..."
3956,Harry Potter and the Sorcerer’s Stone,Action,An alternative cover for this ASIN can be foun...,A 83 Very GoodNotes An effortless enchantment...,4,"[-2.1325233, 0.8182797, -1.8755541, 0.9205673,...","[-1.5902982, 1.1870807, -1.919177, -0.41190028..."
3957,Stormbreaker,Action,They told him his uncle died in an accident He...,I read this book because it is so popular with...,1,"[-1.118215, 2.8452923, -2.2197607, -0.03040268...","[-1.3485644, 2.0394104, -2.8516407, -1.1383078..."
3958,Harry Potter and the Deathly Hallows,Action,Harry has been burdened with a dark dangerous ...,A 86 ExtraordinaryNotes A bittersweet bewitch...,5,"[-1.8149891, 1.2195753, -2.020865, 0.19929937,...","[-1.3487275, 1.4115508, -2.1309614, -0.4195380..."
...,...,...,...,...,...,...,...
4975,Ready Player One,Action,The only time teenage Wade Watts really feels ...,I originally gave this book 3 stars as harmles...,1,"[-1.1675406, 2.0680413, -2.1876495, -0.5904490...","[-1.334821, 1.5825843, -2.2964776, -0.8203, 1...."
4976,Jurassic Park,Action,An astonishing technique for recovering and cl...,Lesson from this book Kids are annoying and wi...,4,"[-0.7587178, 0.15388066, -0.94132376, -1.21109...","[-1.2745761, 1.4290287, -3.3435752, -1.3156902..."
4977,The Angel Experiment,Action,Six unforgettable kids with no families no ho...,Now this is a terrible bookWritten with all th...,1,"[-1.4625356, 2.2057996, -3.4776807, -0.5135589...","[-2.0243204, 1.0571125, -1.7632465, -0.2440451..."
4978,The Lost Symbol,Action,WILL BE FOUNDIn this stunning followup to the ...,I dont get all the haters of the Dan Brown boo...,3,"[-1.679193, 0.8255523, -0.4058209, -0.11516452...","[-1.3919069, 1.4710852, -2.5053828, -1.3461065..."


In [86]:
df = df.iloc[:-1000,:]

In [149]:
X = df.embeddings_title
y = df.genre

In [150]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=1)

In [97]:
# X_train = df.embeddings_description
# y_train = df.genre
# X_test = test.embeddings_description
# y_test = test.genre

In [151]:
X_train = np.stack(X_train)
X_test = np.stack(X_test)

In [152]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [113]:
# from imblearn.under_sampling import RandomUnderSampler
# resampler = RandomUnderSampler()

In [114]:
# X_train,y_train = resampler.fit_resample(X_train,y_train)

In [153]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

clf = RandomForestClassifier()
clf.fit(X_train_scaled,y_train)

RandomForestClassifier()

In [154]:
y_train.value_counts()

genre
Action             660
Mystery            360
Thriller           360
Science-fiction    358
Fiction            355
Fantasy            334
Autobiography      333
Motivational       327
Romance            315
Name: count, dtype: int64

In [155]:
clf.predict(X_test_scaled)

array(['Science-fiction', 'Autobiography', 'Romance', ..., 'Romance',
       'Thriller', 'Action'], dtype=object)

In [156]:
y_test.value_counts()

genre
Action             299
Romance            164
Autobiography      157
Fantasy            144
Motivational       143
Science-fiction    142
Thriller           138
Mystery            137
Fiction            135
Name: count, dtype: int64

In [157]:
from sklearn.metrics import classification_report
print(classification_report(y_test,clf.predict(X_test_scaled)))

                 precision    recall  f1-score   support

         Action       0.71      1.00      0.83       299
  Autobiography       1.00      0.96      0.98       157
        Fantasy       0.93      0.66      0.77       144
        Fiction       0.63      0.46      0.53       135
   Motivational       0.94      1.00      0.97       143
        Mystery       0.83      0.69      0.75       137
        Romance       0.96      0.95      0.95       164
Science-fiction       0.82      0.80      0.81       142
       Thriller       0.82      0.72      0.77       138

       accuracy                           0.83      1459
      macro avg       0.85      0.80      0.82      1459
   weighted avg       0.84      0.83      0.83      1459



In [57]:
df.to_csv('books-embeddings.csv')